In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

# 데이터 읽기 및 처리
data = pd.read_csv("car_evaluation.csv", header=None)
columns = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
data.columns = columns

label_encoder = {}
for column in columns:
    label_encoder[column] = LabelEncoder()
    data[column] = label_encoder[column].fit_transform(data[column])

# 독립 변수와 종속 변수 설정 (예: class를 예측)
X = data.drop("class", axis=1)
y = data["class"]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 텐서 변환
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test.values)

# 회귀 모델 정의
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return self.fc(x)

# 모델 초기화
input_dim = X_train_tensor.shape[1]
model = RegressionModel(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs.view(-1), y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 평가
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).view(-1)
    test_loss = criterion(predictions, y_test_tensor)
    print(f"Test Loss: {test_loss.item():.4f}")


Epoch [10/100], Loss: 3.9154
Epoch [20/100], Loss: 3.4282
Epoch [30/100], Loss: 3.0299
Epoch [40/100], Loss: 2.6983
Epoch [50/100], Loss: 2.4169
Epoch [60/100], Loss: 2.1745
Epoch [70/100], Loss: 1.9643
Epoch [80/100], Loss: 1.7812
Epoch [90/100], Loss: 1.6211
Epoch [100/100], Loss: 1.4810
Test Loss: 1.5021
